In [27]:
import psycopg2 as psql

In [28]:
conn = psql.connect(
    host="localhost",
    database="test",
    user="postgres",
    password="123"
)

In [29]:
cur = conn.cursor()

print('PostgreSQL database version:')

cur.execute('SELECT version()')
db_version = cur.fetchone()
print(db_version)

cur.close()

PostgreSQL database version:
('PostgreSQL 13.2 (Debian 13.2-1.pgdg100+1) on x86_64-pc-linux-gnu, compiled by gcc (Debian 8.3.0-6) 8.3.0, 64-bit',)


In [30]:
conn.close()

## Create tables

In [45]:
percentiles_table = '''
    DROP TABLE IF EXISTS percentiles;
    
    CREATE TABLE percentiles (
        id SERIAL PRIMARY KEY,
        project_id VARCHAR(255) NOT NULL,
        metadata JSON,
        gene_name VARCHAR(255) NOT NULL,
        percentile float NOT NULL,
        number_genes INTEGER NOT NULL,
        number_cells INTEGER NOT NULL
    );
'''

In [46]:
modules_table = '''
    DROP TABLE IF EXISTS modules;

    CREATE TABLE modules (
        id SERIAL PRIMARY KEY,
        name VARCHAR(255) NOT NULL
    );
'''

In [47]:
module_membership_table = '''
    DROP TABLE IF EXISTS module_membership;

    CREATE TABLE module_membership (
        id SERIAL PRIMARY KEY,
        project_id VARCHAR(255) NOT NULL,
        correction VARCHAR(255) NOT NULL,
        iter_pseudocells INTEGER NOT NULL,
        metadata JSON,
        module INTEGER NOT NULL,
        gene_name VARCHAR(255) NOT NULL,
        MM float NOT NULL,
        FOREIGN KEY (module) 
            REFERENCES modules (id) 
            ON UPDATE CASCADE ON DELETE CASCADE
    );
'''

In [48]:
notation_table = '''
    DROP TABLE IF EXISTS notation;

    CREATE TABLE notation (
        id SERIAL PRIMARY KEY,
        name VARCHAR(255) NOT NULL,
        notation_id VARCHAR(255) NOT NULL,
        source VARCHAR(255) NOT NULL,
        IC float NULL
    );
'''

In [49]:
phenotype_notation_table = '''
    DROP TABLE IF EXISTS phenotype_notation;

    CREATE TABLE phenotype_notation (
        id SERIAL PRIMARY KEY,
        name VARCHAR(255) NOT NULL,
        notation_id VARCHAR(255) NOT NULL,
        source VARCHAR(255) NOT NULL
    );
'''

In [50]:
notation_bridge_table = '''
    DROP TABLE IF EXISTS notation_bridge;

    CREATE TABLE notation_bridge (
        module_id INTEGER,
        notation_id INTEGER,
        PRIMARY KEY (module_id, notation_id),
        FOREIGN KEY (module_id) 
            REFERENCES modules (id) 
            ON UPDATE CASCADE ON DELETE CASCADE,
        FOREIGN KEY (notation_id) 
            REFERENCES notation (id) 
            ON UPDATE CASCADE ON DELETE CASCADE
    );
'''

In [51]:
phenotype_notation_bridge_table = '''
    DROP TABLE IF EXISTS phenotype_notation_bridge;

    CREATE TABLE phenotype_notation_bridge (
        module_id INTEGER,
        phenotype_notation_id INTEGER,
        PRIMARY KEY (module_id, phenotype_notation_id),
        FOREIGN KEY (module_id) 
            REFERENCES modules (id) 
            ON UPDATE CASCADE ON DELETE CASCADE,
        FOREIGN KEY (phenotype_notation_id) 
            REFERENCES phenotype_notation (id) 
            ON UPDATE CASCADE ON DELETE CASCADE
    );
'''

In [52]:
commands = [
    percentiles_table,
    modules_table,
    module_membership_table,
    notation_table,
    phenotype_notation_table,
    notation_bridge_table,
    phenotype_notation_bridge_table
]

In [53]:
try:
    # connect to the PostgreSQL server
    conn = psql.connect(
        host="localhost",
        database="test",
        user="postgres",
        password="123"
    )
    cur = conn.cursor()
    
    # create table one by one
    for command in commands:
        cur.execute(command)
    # close communication with the PostgreSQL database server
    cur.close()
    # commit the changes
    conn.commit()
except (Exception, psql.DatabaseError) as error:
    print(error)
finally:
    if conn is not None:
        conn.close()